## 11.2 Movie Recommendation Engine

Joi Chu-Ketterer <br />
DSC650 <br />
February 23, 2020 <br />

This notebook will demonstrate how to build a recommendation system. 

__1. Import Data__

First, the movies and their ratings will be imported as separate files and joined into a single table.

In [2]:
ratings = "/FileStore/tables/ratings.csv"
movies = "/FileStore/tables/movies.csv"

# this imports the data into the workbook
df_ratings = spark.read.format('csv').options(header ='true').load(ratings)
df_movies = spark.read.format('csv').options(header ='true').load(movies)

In [3]:
joined_table = df_ratings.join(df_movies, ["movieId"])
joined_table.printSchema()

root
-- movieId: string (nullable = true)
-- userId: string (nullable = true)
-- rating: string (nullable = true)
-- timestamp: string (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

In [4]:
from pyspark.sql.types import *

joined_table = joined_table.withColumn("userId", joined_table["userId"].cast(IntegerType()))
joined_table = joined_table.withColumn("movieId", joined_table["movieId"].cast(IntegerType()))
joined_table = joined_table.withColumn("rating", joined_table["rating"].cast(DoubleType()))
joined_table = joined_table.withColumn("timestamp",joined_table["timestamp"].cast(IntegerType()))

joined_table.printSchema()

root
-- movieId: integer (nullable = true)
-- userId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: integer (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

__2. Train and Evaluate Recommender__ 

With the data joined in a single table, it can be used to train the recommender algorithm.

In [6]:
train, test = joined_table.randomSplit([0.8, 0.2], seed = 123)

print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 80838
Test Dataset Count: 19998

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS #ALS = alternating least squares
from pyspark.sql import Row

# builds the ASL model
# cold start strategy is set to 'drop' to ensure there are no NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

# this the ASL model to the training data
model = als.fit(train)

# uses RMSE to evaluate the model on the test data 
predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

evaluation_RSME = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(evaluation_RSME))

Root-mean-square error = 1.073022063680764

__3. Generate top 10 movie recommendations__

In [9]:
def get_movieids(userid):
    
    nice = predictions[['prediction', 'movieId', 'userId', 'title']].where(predictions['userId'] == userid)
    
    return nice.sort(['prediction'], reverse = True)

In [10]:
get_movieids(127).show(truncate = False)

+----------+-------+------+----------------------------+
prediction|movieId|userId|title |
+----------+-------+------+----------------------------+
0.22603405|3101 |127 |Fatal Attraction (1987) |
2.2741299 |2455 |127 |Fly, The (1986) |
2.5153782 |5060 |127 |M*A*S*H (a.k.a. MASH) (1970)|
2.918777 |1962 |127 |Driving Miss Daisy (1989) |
4.613855 |55276 |127 |Michael Clayton (2007) |
+----------+-------+------+----------------------------+

In [11]:
get_movieids(151).show(truncate = False)

+----------+-------+------+------------------------------------------+
prediction|movieId|userId|title |
+----------+-------+------+------------------------------------------+
2.106973 |107 |151 |Muppet Treasure Island (1996) |
2.508103 |634 |151 |Theodore Rex (1995) |
2.652124 |95 |151 |Broken Arrow (1996) |
2.6901782 |1047 |151 |Long Kiss Goodnight, The (1996) |
2.6953502 |710 |151 |Celtic Pride (1996) |
2.70832 |1390 |151 |My Fellow Americans (1996) |
2.9764643 |141 |151 |Birdcage, The (1996) |
3.0691888 |719 |151 |Multiplicity (1996) |
3.1498196 |747 |151 |Stupids, The (1996) |
3.702251 |799 |151 |Frighteners, The (1996) |
3.9844193 |648 |151 |Mission: Impossible (1996) |
4.0316167 |62 |151 |Mr. Holland's Opus (1995) |
4.0597973 |104 |151 |Happy Gilmore (1996) |
4.2192802 |724 |151 |Craft, The (1996) |
4.240461 |1073 |151 |Willy Wonka & the Chocolate Factory (1971)|
4.3621874 |1 |151 |Toy Story (1995) |
+----------+-------+------+------------------------------------------+

In [12]:
get_movieids(300).show(truncate = False)

+----------+-------+------+-------------------+
prediction|movieId|userId|title |
+----------+-------+------+-------------------+
2.7763572 |112552 |300 |Whiplash (2014) |
4.3686557 |356 |300 |Forrest Gump (1994)|
4.3908405 |2571 |300 |Matrix, The (1999) |
4.7966957 |2959 |300 |Fight Club (1999) |
+----------+-------+------+-------------------+